In [1]:
import tensorflow as tf
import numpy as np
import os
import sys

In [3]:
from PIL import Image
import time

In [4]:
from six import iteritems, string_types
from matplotlib import pyplot as plt

In [5]:
import pickle as pk

In [6]:
FACE_PIC_SIZE = 160

EMBEDDING_SIZE = 512

PRETREINED_MODEL_DIR = os.path.join(os.getcwd(),'pretrained_models')

UNKNOWN_CLASS = "Mr Nobody"

CLUSTER = "_embeddings.KNN"

In [9]:
padding = ['SAME','VALID']

In [7]:
def load(data_path, session, ignore_missing=False):
    """Load network weights.
    @data_path: The path to the numpy-serialized network weights
    @session: The current TensorFlow session
    @ignore_missing: If true, serialized weights for missing layers are ignored.
    """
    data_dict = np.load(data_path, encoding='latin1').item()  # pylint: disable=no-member

    for op_name in data_dict:
        with tf.variable_scope(op_name, reuse=True):
            for param_name, data in iteritems(data_dict[op_name]):
                try:
                    var = tf.get_variable(param_name)
                    session.run(var.assign(data))
                except ValueError:
                    if not ignore_missing:
                        raise

In [10]:
def conv(inpt, k_h, k_w, c_o, s_h, s_w, name, relu=True, padding=padding[0], group=1, biased=True):
    """
    @inpt: input data to convolve
    @k_h: kernel height  
    @k_w: kernel width
    @c_o: number of filter
    @s_h: strides weight
    @s_w: strides height 
    @name: layer name
    @relu: non linearity RELU
    @padding: filter padding 
    """
# Get the number of channels in the input
    c_i = int(inpt.get_shape()[-1])

    # Convolution for a given input and kernel
    with tf.variable_scope(name) as scope:
        kernel = tf.get_variable('weights', shape=[k_h, k_w, c_i // group, c_o], trainable=True)
        output = tf.nn.conv2d(inpt, kernel, [1, s_h, s_w, 1], padding=padding)
        # Add the biases
        if biased:
            biases = tf.get_variable('biases', [c_o])
            output = tf.nn.bias_add(output, biases)
        if relu:
            # ReLU non-linearity
            output = tf.nn.relu(output, name=scope.name)
        return output

In [11]:
def prelu(inpt, name):
    """
    @inpt: input from previous layer
    @name: layer name
    """
    with tf.variable_scope(name):
        i = int(inpt.get_shape()[-1])
        alpha = tf.get_variable('alpha', shape=(i,))
        output = tf.nn.relu(inpt) + tf.multiply(alpha, -tf.nn.relu(-inpt))
    return output

In [24]:
def max_pool(inpt, k_h, k_w, s_h, s_w, name, padding=padding[0]):
    """
    @inpt: input from previous layer
    @k_h: kernel height
    @k_w: kernel width
    @s_h: stride height
    @s_w: stride width
    @name: layer name
    @padding: Valid Padding ['SAME','VALID']
    """
    return tf.nn.max_pool(inpt, ksize=[1, k_h, k_w, 1],strides=[1, s_h, s_w, 1], padding=padding, name=name)

In [25]:
def fully_connected(inpt, num_out, name, relu=True):
    """
    @inpt: input from previous layer
    @num_out: output layer
    """
    with tf.variable_scope(name):
        input_shape = inpt.get_shape()
        if input_shape.ndims == 4:
            # since the input is spatial.we have to Vectorize it first.
            dim = 1
            for d in input_shape[1:].as_list():
                dim *= int(d)
            feed_in = tf.reshape(inpt, [-1, dim])
        else:
            feed_in, dim = (inpt, input_shape[-1].value)
        weights = tf.get_variable('weights', shape=[dim, num_out])
        biases = tf.get_variable('biases', [num_out])
        op = tf.nn.relu_layer if relu else tf.nn.xw_plus_b
        fc = op(feed_in, weights, biases, name=name)
        return fc

In [26]:
def softmax(target, axis, name=None):
    """
    @target: target layer to apply softmax
    @axis: axes to apply softmax
    @name: layer name
    """
    max_axis = tf.reduce_max(target, axis, keepdims=True)
    target_exp = tf.exp(target - max_axis)
    normalize = tf.reduce_sum(target_exp, axis, keepdims=True)
    softmax = tf.div(target_exp, normalize, name)
    return softmax

In [27]:
def pNet(data):
    value = conv(data, 3, 3, 10, 1, 1, padding='VALID', relu=False, name='conv1')
    value = prelu(value, name='PReLU1')
    value = max_pool(value, 2, 2, 2, 2, name='pool1')
    value = conv(value ,3, 3, 16, 1, 1, padding='VALID', relu=False, name='conv2')
    value = prelu(value ,name='PReLU2')
    value = conv(value ,3, 3, 32, 1, 1, padding='VALID', relu=False, name='conv3')
    value = prelu(value ,name='PReLU3')
#     Face detection classfication probablity output
    val_conv4_1 = conv(value ,1, 1, 2, 1, 1, relu=False, name='conv4-1')
    classify = softmax(val_conv4_1 ,3, name='prob1')
#     bounding box regression
    bbr = conv(value ,1, 1, 4, 1, 1, relu=False, name='conv4-2')
    
    return(classify, bbr)

In [28]:
def rNet(data):
    value = conv(data, 3, 3, 28, 1, 1, padding='VALID', relu=False, name='conv1')
    value = prelu(value, name='prelu1')
    value = max_pool(value, 3, 3, 2, 2, name='pool1')
    value = conv(value, 3, 3, 48, 1, 1, padding='VALID', relu=False, name='conv2')
    value = prelu(value, name='prelu2')
    value = max_pool(value, 3, 3, 2, 2, padding='VALID', name='pool2')
    value = conv(value, 2, 2, 64, 1, 1, padding='VALID', relu=False, name='conv3')
    value = prelu(value, name='prelu3')
    value = fully_connected(value, 128, relu=False, name='conv4')
    value = prelu(value, name='prelu4')
#     face detection classfication probablity output
    conv5_1 = fully_connected(value, 2, relu=False, name='conv5-1')
    classify = softmax(conv5_1, 1, name='prob1')
#     bounding box regression
    bbr = fully_connected(value,4, relu=False, name='conv5-2')
    
    return (classify, bbr)

In [29]:
def oNet(data):
    value = conv(data, 3, 3, 32, 1, 1, padding='VALID', relu=False, name='conv1')
    value = prelu(value, name='prelu1')
    value = max_pool(value ,3, 3, 2, 2, name='pool1')
    value = conv(value, 3, 3, 64, 1, 1, padding='VALID', relu=False, name='conv2')
    value = prelu(value, name='prelu2')
    value = max_pool(value, 3, 3, 2, 2, padding='VALID', name='pool2')
    value = conv(value, 3, 3, 64, 1, 1, padding='VALID', relu=False, name='conv3')
    value = prelu(value, name='prelu3')
    value = max_pool(value, 2, 2, 2, 2, name='pool3')
    value = conv(value, 2, 2, 128, 1, 1, padding='VALID', relu=False, name='conv4')
    value = prelu(value, name='prelu4')
    value = fully_connected(value, 256, relu=False, name='conv5')
    value = prelu(value, name='prelu5')
#     face detection classification
    conv6_1 = fully_connected(value, 2, relu=False, name='conv6-1')
    classify = softmax(conv6_1, 1, name='prob1')
#     bounding box regression
    bbr = fully_connected(value, 4, relu=False, name='conv6-2')
#     face localization
    floc = fully_connected(value, 10, relu=False, name='conv6-3')
    
    return (classify, bbr, floc)

In [30]:
def create_mtcnn(sess, model_path):
    """
    Input @sess: Current session,
    Input @model_path: path of weights of model to load,
    """
#     Creating variable scope for Pnet
    with tf.variable_scope('pnet'):
        data = tf.placeholder(tf.float32, (None, None, None, 3), 'input')
        pnet = pNet(data)
        load(os.path.join(model_path, 'det1.npy'), sess)
        
#     Creating Variabel scope for Rnet
    with tf.variable_scope('rnet'):
        data = tf.placeholder(tf.float32, (None, 24, 24, 3), 'input')
        rnet = rNet(data)
        load(os.path.join(model_path, 'det2.npy'), sess)
        
#    Creating Variable scope for Onet
    with tf.variable_scope('onet'):
        data = tf.placeholder(tf.float32, (None, 48, 48, 3), 'input')
        onet = oNet(data)
        load(os.path.join(model_path, 'det3.npy'), sess)
        
#         Predicting facial features by cascading P-net, R-net and O-net
    pnet_fun = lambda img: sess.run(('pnet/conv4-2/BiasAdd:0', 'pnet/prob1:0'), feed_dict={'pnet/input:0': img})
    rnet_fun = lambda img: sess.run(('rnet/conv5-2/conv5-2:0', 'rnet/prob1:0'), feed_dict={'rnet/input:0': img})
    onet_fun = lambda img: sess.run(('onet/conv6-2/conv6-2:0', 'onet/conv6-3/conv6-3:0', 'onet/prob1:0'), feed_dict={'onet/input:0': img})
    
    return pnet_fun, rnet_fun, onet_fun

In [31]:
def _setup_mtcnn():
    with tf.Graph().as_default():
        sess = tf.Session()
        with sess.as_default():
            return create_mtcnn(sess, PRETREINED_MODEL_DIR)

In [32]:
pnet, rnet, onet = _setup_mtcnn()

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\MLTUT\\Experiments\\pretrained_models\\det1.npy'